# Smoothing
在训练数据中，经过极大似然估计得到的$P(x|c)$为0，但是不能说明在所有数据上$P(x|c)$为0。为了解决这个问题，需要引入Smoothing机制。

## Add-K

既然问题的出现是因为某些gram没有出现在训练预料中，那么我们通过假设其出现过一次可好？拉普拉斯平滑就是这样处理的。对于N-gram
$$
P(w_i|w_1, \ldots, w_{i-1}) = \frac{C(w_1,\ldots, w_i) + 1}{C(w_1, \ldots, w_{i-1}) + |V|}
\tag{1}
$$
公式(1)中的$|V|$代表的是词表的大小。这个词表中应该包含$<end>$。直接加1的好处就是每一个条件概率都不为0，但是由于训练语料的局限性，整个语言L中能够出现的gram数量远远高于语料。这会导致平滑处理之后，所有未出现的gram占据了整个概率分布的一个较大的比例。所以公式(1)变形为下式
$$
P(w_i|w_1, \ldots, w_{i-1}) = \frac{C(w_1,\ldots, w_i) + K}{C(w_1, \ldots, w_{i-1}) + K|V|}
\tag{2}
$$
不一定是加上一个整数1,而是一个K值，但是这个值到底是多少无法确定，通常取值都是小于1的。这个如果将K值作为一个参数，可以通过构造损失函数的方式，再加上优化方法，去逼近最优的K值。

## Backoff

通常认为高阶的N-gram模型更加可靠，但是高阶的模型中出现计数为0也更多，这个时候就应该转而使用低阶模型来避免数据稀疏
$$
P_{backoff} = 
\begin{cases}
P^*(w_i|w_{i-n+1, \ldots, w_{i-1}}) 
\ \ \ \ \ \ \ \ \ \ \  \ \ \ \  \  \ \ \ \ \ \ \ \ \ \  
if\ \ C(w_{i-n+1}, \ldots, w_u) > 0\\
a(w_{i-n+1}, \ldots, w_{i-1})\cdot P_{backoff}(w_i|w_{i-n+2}, \ldots, w_{i-1}), \  otherwise
\end{cases}
\tag{3}
$$

## Interpolation

interpolation的思想是将不同阶别的N-gram模型线性加权组合起来后再使用。Simple Linear Interpolation可以用如下公式定义：
$$
P_{interp}(w_i|w_{i-n+1}, \ldots, w_{i-1}) = \\
\lambda_1P(w_i) + 
\lambda_wP(w_i|w_{i-1}) +
\ldots + 
\lambda_nP(w_i|w_{i-n+1}, \ldots, w_{i-1})
\tag{4}
$$

其中$\lambda_i \ge 0, \sum\lambda_i = 1$,$\lambda$的值可以凭借经验设定，也可以通过应用某些算法确定比如EM。在Simple Linear Interpolation中，不管高阶模型是否可靠，低阶模型均以相同的权值被加入模型，这并不合理，一个解决思路是让$\lambda_i$变成历史的函数，如果使用递归的形式重写插值法的公式，可以表示为如下的形式
$$
P_{interp}(w_i|w_{i-n+1}, \ldots, w_{i-1}) = 
\lambda(w_{i-n+1}, \ldots, w_{i-1})\cdot P(w_i|w_{i-n+1}, \ldots, w_{i-1})\\
+ (1-\lambda(w_{i-n+1}, \ldots, w_{i-1}))\cdot P_{interp}(w_i|w_{i-n+2}, \ldots, w_{i-1})
\tag{5}
$$

## Absolute Discounting

想想之前Add-k算法。本质上说其实是将一些频繁出现的 N-Gram 的概率匀出了一部分，分给那些没有出现的 N-Gram 上。因为所有可能性的概率之和等于1，所以我们只能在各种可能的情况之间相互腾挪这些概率。

既然我们打算把经常出现的一些N-Gram的概率分一些出来，其实也就等同于将它们出现的次数减去（discount）一部分，那到底该discount多少呢？这个值得得出需要统计分析，对于Bigram而言，这个值取0.75是合适的。通常对出现次数大于1的gram进行如下处理。
$$
P_{AbsDiscount} = \frac{C(w_{i-1}w_i) - d}{C(W_{i-1})} + \lambda(w_{i-1})P(w_i) \tag{6}
$$

## Kneser-Ney

这种算法目前是一种标准的，而且是非常先进的平滑算法，它其实相当于是前面讲过的几种算法的综合。由于这个算法比较复杂，我们从一个直观上的例子来开始。

假设我们使用 bigram 和 unigram 的插值模型来预测下面这个句子中空缺的一个词该填什么

- I used to eat Chinese food with ______ instead of knife and fork.

直觉上你一定能猜到这个地方应该填 chopsticks（筷子）。但是有一种情况是训练语料库中，
Zealand 这个词出现的频率非常高，因为 New Zealand 是语料库中高频词。如果你采用标准的
unigram 模型，那么显然 Zealand 会比 chopsticks 具有更高的权重，
所以最终计算机会选择Zealand这个词（而非chopsticks）填入上面的空格，
尽管这个结果看起来相当不合理。这其实就暗示我们应该调整一下策略，最好仅当前一个词是New时，
我们才给 Zealand 赋一个较高的权值，否则尽管在语料库中 Zealand 也是高频词，
但我们并不打算单独使用它。

如果说 $P(w)$衡量了 $w$这个词出现的可能性，那么我们现在想创造一个新的 unigram 模型，叫做 $P_{continuation}$,它的意思是将$w$这个词作为一个新的接续的可能性。注意这其实暗示我们要考虑前面一个词（即历史）的影响。或者说，为了估$P_{continuation}$（注意这是一个 unigram 模型),我们其实需要考察使用了$w$这个词来生成的不同 bigram 的数量。注意这里说使用了$w$这个词来生成的不同类型 bigram 的数量，是指当前词为 $w$,而前面一个词不同时，就产生了不同的类型。例如：$w$ = “food”, 那么不同的 bigram 类型就可能包括 “chinese food”，“english food”，“japanese food”等。每一个 bigram 类型，当我们第一次遇到时，就视为一个新的接续(novel continuation)
$$
P_{continuation}(w_i) \propto |w_{i-1}: C(w_{i-1}w_i) > 0|
\tag{7}
$$

然后，为了公式(7)变成概率，我们需要除以一个比较大的数，这个数所有bigram类型的数量。于是
$$
P_{continuation}(w_i) =\frac{|w_{i-1}: C(w_{i-1}w_i) > 0|}{|(w_{j-1}w_j): C(w_{j-1}w_j) > 0|} 
\tag{8}
$$

对于公式(8)，其分子代表所有当前次为$w$的bigram的种类，分母是预料中所有bigram的种类。如此一来，一个仅出现在New后面的高频词Zealand只能获得一个比较低的续接概率。由此，结合Absolute Discount给出一个概率计算公式
$$
P_{KN}(w_i|w_{i-1}) =\frac{max\big(C(w_{i-1}w_i) -d, 0\big)}{C(w_{i-1})} + \lambda(w_{i-1})P_{continuation}(w_i)
\tag{9}
$$
其中$\lambda$是一个正则化常量，用于分配Discount的概率值。
$$
\lambda(w_{i-1}) = \frac{d}{C(w_{i-1})}\cdot |\{w: C(w_{i-1}, w) > 0\}|
\tag{10}
$$

## Good-Turing Discounting

对于假定在语料库中出现r次的词有$N_r$个。有$N= \displaystyle\sum_{r=1}^\infty rN_r$现在,假定当r比较小时，它的统计可能不可靠，因此在计算那些出现r次的词的概率时，要使用一个更小一点的次数,是dr。古德-图灵估计按照下面的公式计算$d_r$：
$$
d_r = \frac{(r + 1)N_{r + 1}}{N_r}\tag{11}
$$
那么以bigram为例，说明其条件概率的估计公式
$$
P(w_i|w_{i-1}) = \frac{d_r\big[r := C(w_{i-1}w_i)\big]}{C(w_{i-1})}
\tag{12}
$$